In [1]:
import numpy as np
import random 
## b3hash

In [22]:
def okbithash(val,mod=0,mod2=0,mod3=0):
    mod = f'|{mod}|{mod2}|{mod3}|'
    s = val + mod
    h = hash(s)
    return(h%2)

In [3]:
byte_count = np.array([0, 1, 1, 2, 1, 2, 2, 3, 1, 2, 2, 3, 2, 3, 3, 4, 1, 2, 2, 3, 2, 3, 3, 4, 2, 3, 3, 4, 3, 4, 4, 5, 1, 2, 2, 3, 2, 3, 3, 4, 2, 3, 3, 4, 3, 4, 4, 5, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 1, 2, 2, 3, 2, 3, 3, 4, 2, 3, 3, 4, 3, 4, 4, 5, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 3, 4, 4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7, 1, 2, 2, 3, 2, 3, 3, 4, 2, 3, 3, 4, 3, 4, 4, 5, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 3, 4, 4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 3, 4, 4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7, 3, 4, 4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7, 4, 5, 5, 6, 5, 6, 6, 7, 5, 6, 6, 7, 6, 7, 7, 8],dtype=np.uint8)

class gbytes(np.ndarray):
    def __new__(cls,b):
        buf = np.frombuffer(b,dtype=np.uint8)
        qa = np.ndarray.__new__(cls,len(b),dtype=np.uint8)
        qa[:] = buf
        return(qa)
    
    def __repr__(self):
        s = 'g\''
        for b in self:
            s += '\\x{:02x}'.format(b)
        s += '\''
        return(s)

    def bit_count(self):
        count = 0
        for b in self:
            count += byte_count[b]
        return(count)

    @staticmethod
    def similarity(gb1, gb2):
        sim_bin = gbytes(np.bitwise_not( np.bitwise_xor( gb1,gb2 ) ))
        sim = sim_bin.bit_count()
        sim = int(sim)
        return(sim)

        

In [63]:
def b3hash(seq,rounds=256):
    ''' a much bettter version '''
    out_array = np.ndarray(rounds,dtype=np.bool8)
    seq = ["<START>"] + list(seq) + ["<END>"]
    last = True
    bigram_count = dict()
    for round in range(rounds):
        sim_sum = 0
        for i in range(1,len(seq)):
            bigram = (seq[i-1],seq[i])
            bigram_count.setdefault(bigram,0)
            count = bigram_count[bigram]
            h1 = okbithash(seq[i-1],round,mod2=0,mod3=count)
            h2 = okbithash(seq[i],round,mod2=1,mod3=count)
            bigram_count[bigram] += 1
            x12 = h1 ^ h2
            if x12:
                sim_sum += 1
            else:
                sim_sum -= 1
        if sim_sum > 0:
            out_array[round] = True
            last = True
        elif sim_sum < 0:
            out_array[round] = False
            last = False
        else:
            last = (not last)
            out_array[round] = last # deterministic tie-breaker
    out_array = np.packbits(out_array)
    out_array = gbytes(out_array)
    return(out_array)


In [24]:
cats = b3hash('cat')
caats = b3hash('caats')
caaaats = b3hash('caaaats')
catscats = b3hash('catscats')
bcatscats = b3hash('bcatscats')
caattaats = b3hash('caattaats')


In [25]:
elephant = b3hash('elephant')
eleephant = b3hash('eleephant')

In [43]:
alice = b3hash('Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do')
april = b3hash('April was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do')
april2 = b3hash('April was finally tired of sitting by her brother by the tree, and of having nothing to do')
ana = b3hash('of sitting by her nothing to do the bank, and of having Ana was beginning sister on to get very tired ')
#ana = b3hash('Ana was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do')


long_sentence = b3hash('This is an extremely long sentence which should result in similarity for some reason ok bye')
longsentence = b3hash('Thisisanextremelylongsentencewhichshouldresultinsimilarityforsomereasonokbyesomemorewordshere')

shadowsfollow = b3hash('shadowsfollow')

In [44]:
garbage = b3hash('qwertysjdfniabuioopkjhgfdfvbsdljfnjkasdnxcvbkxjcvdfkjnndmmbyasdkjfbkhbsdfkzxcvibekxcbvkasdljfnkjksbdf')


In [61]:
pow = b3hash('aaaaab')
wow = b3hash('aaaaac')

In [45]:
garbage2 = b3hash(str([random.randint(87,122) for _ in range(5)]))

In [62]:
gbytes.similarity(pow,wow) / 256

1.0

In [46]:
gbytes.similarity(alice,april) / 256

0.91796875

In [47]:
gbytes.similarity(alice,april2) / 256

0.65234375

In [48]:
gbytes.similarity(alice,ana) / 256

0.83984375

In [49]:
gbytes.similarity(april,garbage2) / 256

0.48046875

In [50]:
gbytes.similarity(long_sentence,april) / 256

0.53125

In [51]:
gbytes.similarity(long_sentence,alice) / 256

0.52734375

In [52]:
gbytes.similarity(garbage,april) / 256

0.5234375

In [53]:
gbytes.similarity(longsentence,long_sentence) / 256

0.66796875

In [54]:
gbytes.similarity(garbage,longsentence) / 256

0.44140625

In [55]:
gbytes.similarity(caattaats,shadowsfollow) / 256

0.55078125

In [56]:
gbytes.similarity(garbage2,april) / 256

0.48046875

In [57]:
garbage2

g'\x7a\xd9\x3e\xcd\x6c\x1e\x31\x0c\xe6\x00\x71\x92\xb4\x51\x71\x0c\xeb\x21\xcf\xb9\xb8\x64\x00\x07\xb1\x40\xcf\x41\xf0\xe8\xf4\x3b'